# ゼロから作る Deep Learning 4 強化学習編 勉強ノート 第4章 〜動的計画法〜

ここでは、ベルマン方程式によって得られる連立方程式を解くための「<font color="red">**動的計画法**</font>」について扱う。

## 動的計画法と方策評価

強化学習の問題の多くは2つのタスクに取り組むことになる。  
ある方策 $\pi$ が与えられたときに、その方策の価値関数 $v_\pi(s)$ や $q_\pi(s, a)$ を求めることを<font color="red">**方策評価**</font>(Policy Evaluation)といい、方策を制御して最適方策へと調整することを<font color="red">**方策制御**</font>(Policy Control)という。  
強化学習の最終的なゴールは方策制御だが、そのための最初のステップとして方策評価に取り組むことは多くある。  
ここでは、動的計画法(Dynamic Programming)で方策評価を行う。  
以降、動的計画法を DP と略記する。

### DP の概要

以下の価値関数は無限を含む期待値の計算をしているため、通常は計算できない。

$$v_{\pi}(s) = \mathbb{E}_\pi[R_t + \gamma R_{t+1} + \gamma^2 R_{t+2} + \cdots| S_t = s]$$

これを解決するのがベルマン方程式である。

$$v_{\pi}(s) = \sum_{a, s'} \pi(a|s) p(s'|s, a) \left\{ r(s, a, s') + \gamma v_{\pi}(s') \right\}$$

これは、「現在の状態 $s$ の価値関数 $v_{\pi}(s)$」と「次の状態 $s'$ の価値関数 $v_{\pi}(s')$」との関係性を表している。  
これを「更新式」として見ると

$$V_{k+1}(s) = \sum_{a, s'} \pi(a|s) p(s'|s, a) \left\{ r(s, a, s') + \gamma V_k(s') \right\}$$

のようになる。  
$V$ は推測値であり、真の価値関数 $v$ とは異なる。  
ここで行われているのは、「推定値 $V_k(s')$」を使って「別の推定値 $V_{k+1}(s)$」を改善することであり、このプロセスを<font color="red">**ブートストラッピング**</font>(Bootstrapping)という。

### 反復方策評価(*コーディング*)

初期値 $V_0(s)$ から $V_1(s)$ へ更新し、さらに $V_2(s)$ へ更新というように繰り返し行うことで $v_{\pi}(s)$ へと近づいていくアルゴリズムを<font color="red">**反復方策評価**</font>(Iterative Policy Evaluation)という。  

In [ ]:
# ライブラリのインポート
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# グリッドワールドを作成する関数を定義
# 0: 進める道, 1: 障害物, 2: スタート, 3: ゴール, 4: 加点ポイント, 5: 減点ポイント
def create_grid_world(
        rows: int, cols: int,
        start: set, goal: set,
        obstacles: list, items: dict, enemies: dict
    ):
    grid = np.zeros((rows, cols))
    points = np.zeros((rows, cols), dtype=int)

    for obs in obstacles:
        grid[obs] = 1

    grid[start] = 2
    if goal != ():
        grid[goal] = 3

    for item, point in items.items():
        grid[item] = 4
        points[item] = point

    for enemy, point in enemies.items():
        grid[enemy] = 5
        points[enemy] = point

    return grid, points


# グリッドワールドを描画する関数を定義
def draw_grid_world(grid, points, figsize_x, figsize_y):
    rows, cols = grid.shape
    fig = plt.figure(figsize=(figsize_x, figsize_y), tight_layout=True)
    ax = fig.subplots()

    ax.matshow(grid, vmin=-0.5, vmax=5)

    ax.set_xticks(np.arange(-.5, cols, 1), minor=True)
    ax.set_yticks(np.arange(-.5, rows, 1), minor=True)
    ax.grid(which="minor", color="black", linestyle="-", linewidth=2)

    for (i, j), val in np.ndenumerate(grid):
        if val == 1:
            ax.text(
                j, i, "X", ha="center", va="center",
                color="yellow", fontsize=16, fontweight="bold"
            )
        elif val == 2:
            ax.text(
                j, i, "S", ha="center", va="center",
                color="red", fontsize=16, fontweight="bold"
            )
        elif val == 3:
            ax.text(
                j, i, "G", ha="center", va="center",
                color="blue", fontsize=16, fontweight="bold"
            )
        elif val == 4:
            ax.text(
                j, i, f"+{points[i, j]}", ha="center", va="center",
                color="white", fontsize=16, fontweight="bold"
            )
        elif val == 5:
            ax.text(
                j, i, f"{points[i, j]}", ha="center", va="center",
                color="black", fontsize=16, fontweight="bold"
            )

    ax.set_xticks([])
    ax.set_yticks([])

    plt.show()

In [ ]:
# グリッドワールドのパラメータを設定
rows, cols = 1, 4
start = (0, 1)
goal = ()
obstacles = []
items = {(0, 2): +1}
enemies = {(0, 0): -1, (0, 3): -1}

# グリッドワールドを作成して描画
grid, points = create_grid_world(
    rows, cols, start, goal, obstacles, items, enemies
)
draw_grid_world(grid, points, cols, rows)

2マスのグリッドワールド問題において、エージェントはランダムな方策 $\pi$ (確率0.5で左へ移動、確率0.5で右へ移動)に従って行動すると仮定する。  
なお、状態遷移は決定論的に決まるとする。  
このとき、価値関数の更新式は次のように簡略化できる。

$$
V_{k+1}(s) = \sum_{a, s'} \pi(a|s) p(s'|s, a) \left\{ r(s, a, s') + \gamma V_k(s') \right\} \\
\downarrow \space \downarrow \space \downarrow \\
V_{k+1}(s) = \sum_a \pi(a|s) \sum_{s'} p(s'|s, a) \left\{ r(s, a, s') + \gamma V_k(s') \right\} \\
\downarrow \space \downarrow \space \downarrow \\
V_{k+1}(s) = \sum_a \pi(a|s) \left\{ r(s, a, s') + \gamma V_k(s') \right\}
$$

最後の式は、状態遷移は決定論的であることから $s' = f(s, a)$ として $\sum_{s'} p(s'|s, a)$ の部分を省略している。

方策 $\pi$ における価値関数を反復方策評価アルゴリズムを使って求めてみる。  
まずは初期値として $V_0(s) = 0$ とする。  
すなわち、

\begin{eqnarray*}
V_0((0, 1)) &=& 0 \\
V_0((0, 2)) &=& 0
\end{eqnarray*}

となる。  
状態(0, 1)にいるとき、割引率 $\gamma$ を0.9とすると更新式は

$$
0.5 \left \{ -1 + 0.9 V_0((0, 1)) \right \} \\
0.5 \left \{ 1 + 0.9 V_0((0, 2)) \right \}
$$

となるので、 $V_1((0, 1))$ は

\begin{eqnarray*}
V_1((0, 1)) &=& 0.5 \left \{ -1 + 0.9 V_0((0, 1)) \right \} + 0.5 \left \{ 1 + 0.9 V_0((0, 2)) \right \} \\
&=& 0.5 \left( -1 + 0.9 \cdot 0 \right) + 0.5 \left( 1 + 0.9 \cdot 0 \right) \\
&=& 0
\end{eqnarray*}

となる。  
同様にして $V_1((0, 2))$ は

\begin{eqnarray*}
V_1((0, 2)) &=& 0.5 \left \{ 0 + 0.9 V_0((0, 1)) \right \} + 0.5 \left \{ -1 + 0.9 V_0((0, 2)) \right \} \\
&=& 0.5 \left( 0 + 0.9 \cdot 0 \right) + 0.5 \left( -1 + 0.9 \cdot 0 \right) \\
&=& -0.5
\end{eqnarray*}

となる。  
これをプログラムで繰り返していく。

In [ ]:
# V の初期化
V = {"s_1": 0.0, "s_2": 0.0}
new_V = V.copy()

# V を new_V を使って100回更新
for _ in range(100):
    new_V["s_1"] = 0.5 * (-1 + 0.9 * V["s_1"]) +\
                   0.5 * (1 + 0.9 * V["s_2"])
    new_V["s_2"] = 0.5 * (0 + 0.9 * V["s_1"]) +\
                   0.5 * (-1 + 0.9 * V["s_2"])

    V = new_V.copy()
    print(V)

真の価値関数の値は

```
{'s_1': -2.25, 's_2': -2.75}
```

であるので、上の実験結果はこれとほぼ一致していることがわかる。  
今度は、閾値を設定して更新する回数を自動で決める。

In [ ]:
# V の初期化
V = {"s_1": 0.0, "s_2": 0.0}
new_V = V.copy()

# カウントしながら閾値を超えるまで更新する
cnt = 0
while True:
    new_V["s_1"] = 0.5 * (-1 + 0.9 * V["s_1"]) + 0.5 * (1 + 0.9 * V["s_2"])
    new_V["s_2"] = 0.5 * (0 + 0.9 * V["s_1"]) + 0.5 * (-1 + 0.9 * V["s_2"])

    delta = abs(new_V["s_1"] - V["s_1"])
    delta = max(delta, abs(new_V["s_2"] - V["s_2"]))

    V = new_V.copy()

    cnt += 1
    if delta < 0.0001:
        print(V)
        print(cnt)
        break

### 反復方策評価の別の方法(*コーディング*)

先ほどの方法とは別に、辞書の各要素を上書きする方法を扱う。  
これを「上書き方式」と呼ぶことにする。

In [ ]:
# V の初期化
V = {"s_1": 0.0, "s_2": 0.0}

# カウントしながら「上書き方式」で閾値を超えるまで更新する
cnt = 0
while True:
    t = 0.5 * (-1 + 0.9 * V["s_1"]) + 0.5 * (1 + 0.9 * V["s_2"])
    delta = abs(t - V["s_1"])
    V["s_1"] = t

    t = 0.5 * (0 + 0.9 * V["s_1"]) + 0.5 * (-1 + 0.9 * V["s_2"])
    delta = max(delta, abs(t - V["s_2"]))
    V["s_2"] = t

    cnt += 1
    if delta < 0.0001:
        print(V)
        print(cnt)
        break

今回は `V` という辞書のみを使い、各要素が即座に上書きされるようにした。  
前回が76回だったのに対し、今回は60回と少なくなっていることが確認できる。  
以降、反復方策評価アルゴリズムは「上書き方式」で実装する。

## より大きな問題へ

ここでは、「2マスのグリッドワールド」から「3×4のグリッドワールド」に問題の大きさをグレードアップする。  
問題の設定は次のとおりである。

- エージェントは上下左右の4方向に移動できる
- "X" で表されたマス目は壁を表し、壁には入れない
- グリッドワールドは壁で囲われており、それ以上先に進むことはできない
- 壁にぶつかった場合の報酬は0とする
- 環境の状態遷移は一意に定まる
- エピソードタスクとして、報酬 $+1$ を取ったら終了とする

### GridWorld クラスの実装(*コーディング*)

ここでは `GridWorld` クラスを実装する。

In [ ]:
# グリッドワールドのパラメータを設定
rows, cols = 5, 6
start = (3, 1)
goal = ()
obstacles = [
    (0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5),
    (1, 0), (1, 5), (2, 0), (2, 2), (2, 5), (3, 0),
    (3, 5), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5)
]
items = {(1, 4): +1}
enemies = {(2, 4): -1}

# グリッドワールドを作成して描画
grid, points = create_grid_world(
    rows, cols, start, goal, obstacles, items, enemies
)
draw_grid_world(grid, points, cols, rows)

上図を探索する `GridWorld` クラスを作成する。

In [ ]:
# GridWorld クラスの実装
class GridWorld:
    def __init__(self):
        self.action_space = [0, 1, 2, 3]
        self.action_meaning = {
            0: "UP",
            1: "DOWN",
            2: "LEFT",
            3: "RIGHT"
        }
        self.reward_map = np.array(
            [
                [0, 0, 0, 1.0],
                [0, None, 0, -1.0],
                [0, 0, 0, 0]
            ]
        )
        self.goal_state = (0, 3)
        self.wall_state = (1, 1)
        self.start_state = (2, 0)
        self.agent_state = self.start_state

    @property
    def height(self):
        return len(self.reward_map)

    @property
    def width(self):
        return len(self.reward_map[0])

    @property
    def shape(self):
        return self.reward_map.shape

    def actions(self):
        return self.action_space

    def states(self):
        for h in range(self.height):
            for w in range(self.width):
                yield (h, w)

    def next_state(self, state, action):
        action_move_map = [
            (-1, 0), (1, 0), (0, -1), (0, 1)
        ]
        move = action_move_map[action]
        next_state = (state[0] + move[0], state[1] + move[1])
        ny, nx = next_state

        if nx < 0 or nx >= self.width or ny < 0 or ny >= self.height:
            next_state = state
        elif next_state == self.wall_state:
            next_state = state

        return next_state

    def reward(self, state, action, next_state):
        return self.reward_map[next_state]

ここでは `@property` デコレータを使って `GridWorld` クラスに便利なインスタンス変数をいくつか実装している。  
これは、メソッド名の前行に配置することで対象のメソッドをインスタンス変数として使用することができる。

In [ ]:
# GridWorld クラスのインスタンスを生成
env = GridWorld()

# デコレータを使ったメソッドをインスタンス変数として表示できることの確認
print(env.height)
print(env.width)
print(env.shape)

また、 `GridWorld` クラスの `actions()` と `states()` というメソッドを使うことで、すべての行動、すべての状態に対して順にアクセスできる。

In [ ]:
# actions メソッドからすべての行動を出力
for action in env.actions():
    print(action)

# 分ける
print("=================")

# states メソッドからすべての状態を出力
for state in env.states():
    print(state)

続いて、環境の状態遷移を表すメソッド `next_state()` と報酬関数のメソッド `reward()` を確認する。

`next_state()` メソッドの前半では、壁やグリッドワールドの枠は一旦無視して移動先の計算をする。  
後半で枠外や壁に移動していないか判定し、移動が出来なければ現在の状態のままとする。

`reward()` メソッドは、報酬関数の数式の $r(s, a, s')$ の引数に対応させるためにこのような引数を使っている。

### defaultdict を使う(*コーディング*)

辞書型の変数の場合、 `key` に相当するものが存在しない場合にエラーが発生する。  
`collections` ライブラリの `defaultdict` を使うと辞書内に存在しない `key` にアクセスしても自動で生成してくれる。

In [ ]:
# ライブラリのインポート
from collections import defaultdict

In [ ]:
# GridWorld のインスタンスを生成し、辞書 V を defaultdict で生成
env = GridWorld()
V = defaultdict(lambda: 0)

# 辞書の中にない state を key に取って実行する
state = (1, 2)
print(V[state])

では、ランダムな方策を `defaultdict` で実装する。  
各行動 `[0, 1, 2, 3]` が均一にランダムに行われる場合、それぞれの行動が取られる確率は $\frac{1}{4}$ である。  
これを踏まえて、ランダムな方策は次のように実装できる。

In [ ]:
# 行動の確率を defaultdict の辞書に落とし込む
pi = defaultdict(
    lambda: {
        0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25
    }
)

# 辞書の中にない state を key に取って実行する
state = (0, 1)
print(pi[state])

### 反復方策評価の実装(*コーディング*)

ここでは、反復方策評価アルゴリズムを実装する。  
まずは、1ステップの更新だけを行うメソッドを実装する。

In [ ]:
# 1ステップの更新を行う関数を定義
def eval_onestep(
        pi: dict, V: dict, env: GridWorld, gamma: float = 0.9
    ) -> dict:
    for state in env.states():
        if state == env.goal_state:
            V[state] = 0
            continue

        action_probs = pi[state]
        new_V = 0

        for action, action_prob in action_probs.items():
            next_state = env.next_state(state, action)
            r = env.reward(state, action, next_state)

            new_V += action_prob * (r + gamma * V[next_state])

        V[state] = new_V

    return V

これで価値関数の1ステップの更新が終わるので、この更新を繰り返すメソッドを実装する。

In [ ]:
# 方策評価を行う関数を定義
def policy_eval(
        pi: dict, V: dict, env: GridWorld, gamma: float, threshold: float=0.001
    ) -> dict:
    while True:
        old_V = V.copy()
        V = eval_onestep(pi, V, env, gamma)

        delta = 0
        for state in V.keys():
            t = abs(V[state] - old_V[state])
            if delta < t:
                delta = t

        if delta < threshold:
            break

    return V

これと `GridWorld` クラスを使って方策評価を行ってみる。

In [ ]:
# パラメータの初期化
env = GridWorld()
gamma = 0.9
pi = defaultdict(
    lambda: {
        0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25
    }
)
V = defaultdict(lambda: 0)

# 方策評価を行う
V = policy_eval(pi, V, env, gamma)
print(V)

それぞれの位置でのランダムな方策における価値関数が分かった。  
例えば(2, 0)の位置では-0.0989となっており、これはスタート地点からランダムに動くと得られる収益の期待値は-0.0989になるということである。  
全体的に見ると、(0, 0), (0, 1), (0, 2), (0, 3)と一番上の行以外が負の値を取っており、(1, 3)にある負の報酬の影響を強く受けているということがわかる。

DP を使うことで効率的に方策の評価が行える。  
しかし、これはまだ方策の評価だけしか行っていないので、次は最適方策を見つける方法について考える。

## 方策反復法

我々の目標は最適方策を得ることであり、そのための1つの方法がベルマン方程式を満たす連立方程式を解くことであった。  
しかし、その方法には計算量的に問題があり、状態のサイズを $S$ 、行動のサイズを $A$ としたときに、解を求めるために $A^S$ のオーダーの計算量が必要になる。

前節では、 DP を使って方策を評価した。  
それは「反復方策評価」と呼ばれるアルゴリズムであり、方策を「評価」できた。  
次は、方策の「改善」をする。

### 方策の改善

次の記号を用いて方策の改善をする方法について説明する。

- 最適方策: $\mu_*(s)$
- 最適方策における状態価値関数: $v_*(s)$
- 最適方策における行動価値関数: $q_*(s, a)$

まずは復習として、最適方策 $\mu_*$ は次の式で表された。

\begin{eqnarray*}
\newcommand{\argmax}{\mathop{\rm arg~max}\limits}
\mu_*(s) &=& \argmax_a q_*(s, a) \\
&=& \argmax_a \sum_{s'} p(s'|s, a) \left\{ r(s, a, s') + \gamma v_*(s') \right\}
\end{eqnarray*}

これは、最適方策 $\mu_*$ に対しての式であるが、ここでは「何らかの決定論的方策 $\mu$」に対して上記の式を適用することを考える。

\begin{eqnarray*}
\newcommand{\argmax}{\mathop{\rm arg~max}\limits}
\mu'(s) &=& \argmax_a q_\mu(s, a) \\
&=& \argmax_a \sum_{s'} p(s'|s, a) \left\{ r(s, a, s') + \gamma v_\mu(s') \right\}
\end{eqnarray*}

ここでは、

- 現状の方策: $\mu(s)$
- 方策 $\mu(s)$ における状態価値関数: $v_\mu(s)$
- 新たな方策: $\mu'(s)$

で表記する。  
またこの式で方策を更新することを「greedy 化」と呼ぶことにする。  
この greedy 化された方策 $\mu'(s)$ にある性質の1つとして、もしすべての状態 $s$ において $\mu(s)$ と $\mu'(s)$ が同じであれば、方策 $\mu(s)$ は既に最適方策であると言える。  
なぜなら、もし

$$
\newcommand{\argmax}{\mathop{\rm arg~max}\limits} \\
\mu'(s) = \argmax_a q_\mu(s, a)
$$

によって方策が変更されなければ、次の式を満たすことになるからである。

$$
\newcommand{\argmax}{\mathop{\rm arg~max}\limits} \\
\mu(s) = \argmax_a q_\mu(s, a)
$$

この式は最適方策が満たす式そのものである。  
そのため、 greedy 化を行ってもすべての状態 $s$ において $\mu'(s)$ が更新されないのであれば、 $\mu(s)$ は既に最適方策だということになる。  
方策 $\mu'$ と方策 $\mu$ が異なる場合、すべての状態 $s$ において $v_{\mu'}(s) \geq v_\mu(s)$ が成り立つ。  
以上をまとめると、

- 方策は常に改善される
- もし方策の改善がなければそれが最適方策である

ということである。

### 評価と改善を繰り返す

ここでは状態価値関数に基づく式

$$
\newcommand{\argmax}{\mathop{\rm arg~max}\limits}
\mu'(s) = \argmax_a \sum_{s'} p(s'|s, a) \left\{ r(s, a, s') + \gamma v_\mu(s') \right\}
$$

を想定して話を進める。  
我々は前節で状態価値関数を評価するアルゴリズムを実装した。  
この方法の処理の流れは次のようになる。

- まずは $\pi_0$ という方策からスタートする(確率的方策も考えられるので $\mu_0(s)$ ではなく $\pi_0(s|a)$ と表記する)
- 反復方策評価アルゴリズムにより、方策 $\pi_0$ における価値関数を評価して $V_0$ を得る
- 価値関数 $V_0$ を使って greedy 化を行い、決定論的な方策 $\mu_1$ を得る

このフローを繰り返すことで、 greedy 化によって方策が変更されない地点に到達する。  
この評価と改善を繰り返すアルゴリズムを<font color="red">**方策反復法**</font>(Policy Iteration)という。

## 方策反復法の実装

前と同じ「3×4のグリッドワールド」問題を扱う。  
目標は、方策反復法を使って最適方策を得ることである。

### 方策の改善(*コーディング*)

方策を改善するには、現状の価値関数に対して greedy な方策を得る。

$$
\newcommand{\argmax}{\mathop{\rm arg~max}\limits}
\mu'(s) = \argmax_a \sum_{s'} p(s'|s, a) \left\{ r(s, a, s') + \gamma v_\mu(s') \right\}
$$

また、今回の問題では状態は一意に遷移する。  
そのため、方策の greedy 化は

$$
\newcommand{\argmax}{\mathop{\rm arg~max}\limits}
\mu'(s) = \argmax_a \left\{ r(s, a, s') + \gamma v_\mu(s') \right\}
$$

と簡略化できる。

In [ ]:
# 辞書を受け取り、辞書の値が最大となる key を返す関数を定義
def argmax(d: dict) -> int:
    max_value = max(d.values())
    max_key = 0

    for key, value in d.items():
        if value == max_value:
            max_key = key

    return max_key

In [ ]:
# 行動価値の辞書を設定
action_values = {0: 0.1, 1: -0.3, 2: 9.9, 3: -1.3}

# argmax 関数で行動を獲得
max_action = argmax(action_values)
print(max_action)

この `argmax` 関数を使って、価値関数を greedy 化する関数を実装する。

In [ ]:
# 価値関数を greedy 化する関数を定義
def greedy_policy(
        V: dict, env: GridWorld, gamma: float
    ) -> dict:
    pi = {}

    for state in env.states():
        action_values = {}

        for action in env.actions():
            next_state = env.next_state(state, action)
            r = env.reward(state, action, next_state)
            value = r + gamma * V[next_state]
            action_values[action] = value

        max_action = argmax(action_values)
        action_probs = {0: 0, 1: 0, 2: 0, 3: 0}
        action_probs[max_action] = 1.0
        pi[state] = action_probs

    return pi

### 評価と改善を繰り返す(*コーディング*)

評価と改善を繰り返す「方策反復法」の実装準備が整った。  
ここでは、方策反復法を `policy_iter()` という関数で実装する。  
引数は以下の通りである。

- `env`: 環境
- `gammma`: 割引率
- `threshold`: 方策評価を行うときの更新を止めるための閾値

In [ ]:
# 方策反復法の関数を定義
def policy_iter(
        env: GridWorld, gamma: float, threshold: float=0.001
    ) -> dict:
    pi = defaultdict(
        lambda: {
            0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25
        }
    )
    V = defaultdict(lambda: 0)

    while True:
        V = policy_eval(pi, V, env, gamma, threshold)
        new_pi = greedy_policy(V, env, gamma)

        if new_pi == pi:
            break

        pi = new_pi

    return pi

In [ ]:
# パラメータの設定
env = GridWorld()
gamma = 0.9

# 方策反復法で解く
pi = policy_iter(env, gamma)
print(pi)

これにて方策反復法を使って最適方策に辿り着くことができた。

## 価値反復法

方策反復法は、「評価」と「改善」を交互に繰り返す。  
「評価」のフェーズでは、方策 $\mu$ を評価して $V_\mu$ を得る。  
一方で、「改善」のフェーズでは、 $V$ を greedy 化する。

実は、「評価」と「改善」という2つの作業を交互に繰り返す枠組みでは、「評価」を完全に行う前に「改善」フェーズに行ったり、「改善」を完全に行う前に「評価」フェーズへ行ったりする<font color="red">**一般化方策反復**</font>(Generalized Policy Iteration)という概念がある。  
方策反復法では、「評価」と「改善」をそれぞれ最大限に行って、交互にフェーズを切り替えたが、これを最小限に抑えるのが<font color="red">**価値反復法**</font>(Value Iteration)のアイデアになる。

### 価値反復法の導出

方策反復法では、すべての状態における価値関数を何度も更新し、それが収束してから greedy 化のフェーズに移った。  
これと対極にある更新方法は、1つの状態だけを一度更新したらすぐに greedy 化のフェーズに移るというものである。  
このアイデアを数式を使って整理する。  
改善のフェーズで行う greedy 化は次のように表せた。

$$
\newcommand{\argmax}{\mathop{\rm arg~max}\limits}
\mu(s) = \argmax_a \sum_{s'} p(s'|s, a) \left\{ r(s, a, s') + \gamma V(s') \right\}
$$

ここでは、現状の価値関数を $V(s)$ で表すことにする。  
続いて評価のフェーズだが、更新前の価値関数を $V(s)$ 、更新後の価値関数を $V'(s)$ とすれば DP による更新式は次のように表される。

$$V'(s) = \sum_{a, s'} \pi(a|s) p(s'|s, a) \left\{ r(s, a, s') + \gamma V(s') \right\}$$

方策 $\pi(a|s)$ が確率的な方策として表記されている。  
ただし、「改善」のフェーズを一度経由することで方策は greedy な方策として表すことができる。  
そのため、決定論的な方策 $\mu(s)$ として扱うことができ、次のように簡略化できる。

$$V'(s) = \sum_{s'} p(s'|s, a) \left\{ r(s, a, s') + \gamma V(s') \right\}$$

これが「評価」フェーズにおける価値関数の更新式である。

2つの式

$$
\newcommand{\argmax}{\mathop{\rm arg~max}\limits}
\mu(s) = \argmax_a \sum_{s'} p(s'|s, a) \left\{ r(s, a, s') + \gamma V(s') \right\} \\
V'(s) = \sum_{s'} p(s'|s, a) \left\{ r(s, a, s') + \gamma V(s') \right\}
$$

では

$$\sum_{s'} p(s'|s, a) \left\{ r(s, a, s') + \gamma V(s') \right\}$$

の部分で同じ計算を行っていることがわかるので、1つにまとめると

$$V'(s) = \max_a \sum_{s'} p(s'|s, a) \left\{ r(s, a, s') + \gamma V(s') \right\}$$

となる。  
ここで注目すべき点は、方策 $\mu$ が登場しないことである。  
方策を使わずに価値関数を更新しているので、最適価値関数を得るアルゴリズムは「価値反復法」と呼ばれる。  
また、ベルマン最適方程式

$$v_*(s) = \max_a \sum_{s'} p(s'|s, a) \left\{ r(s, a, s') + \gamma v_*(s') \right\}$$

と比べると、これはベルマン最適方程式を「更新式」にしたものであることがわかる。  
さらに、この更新式は次のようにも表せる。

$$V_{k+1}(s) = \max_a \sum_{s'} p(s'|s, a) \left\{ r(s, a, s') + \gamma V_k(s') \right\}$$

価値反復法は無限回更新すると最適価値関数が得られるが、現実的にはどこかで止める必要がある。  
あらかじめ閾値を決めておき、すべての状態の更新量がその閾値を下回った場合に更新を止めるようにする。  
なお、 $V_*(s)$ が得られれば、最適方策 $\mu_*(s)$ は次のように得られる。

$$
\newcommand{\argmax}{\mathop{\rm arg~max}\limits}
\mu_*(s) = \argmax_a \sum_{s'} p(s'|s, a) \left\{ r(s, a, s') + \gamma V_*(s') \right\}
$$

### 価値反復法の実装(*コーディング*)

価値反復法の実装を行う。  
ここでも「3×4のグリッドワールド」問題を扱う。  
この問題では状態遷移は決定論的なので、価値関数の更新式は次のように簡略化できる。

$$V'(s) = \max_a \left\{ r(s, a, s') + \gamma V(s') \right\}$$

まずはこの式によって一度だけ更新する関数を実装する。

In [ ]:
# 一度だけ更新する関数の定義
def value_iter_onestep(
        V: dict, env: GridWorld, gamma: float
    ) -> dict:
    for state in env.states():
        if state == env.goal_state:
            V[state] = 0
            continue

        action_values = []
        for action in env.actions():
            next_state = env.next_state(state, action)
            r = env.reward(state, action, next_state)
            value = r + gamma * V[next_state]
            action_values.append(value)

        V[state] = max(action_values)

    return V

この `value_iter_onestep` 関数を更新が収束するまで繰り返し呼ぶ関数 `value_iter` を実装する。

In [ ]:
# 更新を繰り返す関数を定義
def value_iter(
        V: dict, env: GridWorld, gamma: float, threshold: float=0.001
    ) -> dict:
    while True:
        old_V = V.copy()
        V = value_iter_onestep(V, env, gamma)

        delta = 0
        for state in V.keys():
            t = abs(V[state] - old_V[state])
            if delta < t:
                delta = t

        if delta < threshold:
            break

    return V

それではこれを使って見てみる。

In [ ]:
# パラメータの設定
V = defaultdict(lambda: 0)
env = GridWorld()
gamma = 0.9

# 価値反復法で更新を繰り返す
V = value_iter(V, env, gamma)

# 価値関数を greedy 化する
pi = greedy_policy(V, env, gamma)
print(pi)

価値関数は最初は全てが0の要素を持つ辞書からスタートする。  
そこから更新を繰り返して価値関数の値が収束する。  
これで価値反復法を使って効率的に最適方策を得ることができた。